In [68]:
import math

In [77]:
def B1(thickness, TVL): # Calculates B factor using only 1 TVL. Important for PATIENT SCATTER
    '''
    param thickness: thickness of barrier (cm)
    param TVL: TVL (cm) where it is ALWAYS dependent on energy. Can also depend on the angle too
    '''
    B = 10**-(thickness/TVL)
    return B

def B2(thickness, TVL1, TVLe): # Calculates B factor using 2 TVLs. Important for LEAKAGE
    '''
    param thickness: thickness of barrier (cm)
    param TVL1: first TVL (cm)
    param TVLe: equilibrium TVL (cm)
    '''
    B = 10**-(1+((thickness-TVL1)/TVLe))
    return B

def H_ps_unshielded(W, d_secondary, d_scatter, a, F): # Calculates unshielded equivalent dose for PATIENT SCATTER
    '''
    param W: workload in a typical week (Gy/wk)
    param d_secondary: distance from isocenter to the point of interest (cm). Includes the 0.3 m
    param d_scatter: distance from target to isocenter. Usually 1 m
    param a: scatter fraction dependent on angle (deg) and beam energy (MV)
    param F: field size in cm^2
    '''
    H_ps_uns = (W * a * F) / (d_secondary**2 * d_scatter**2 * 400) * 10**3
    return H_ps_uns # (mSv/wk)

def H_leak_unshielded(W_L, d_leakage): # Calculates UNSHIELDED equivalent dose for LEAKAGE
    '''
    param W_L: leakage workload (Gy/wk)
    param d_leakage: distance from source of leakage to point of protection (cm)
    '''
    H_leak_uns = ((W_L*10**-3) / d_leakage**2) * 10**3 # 10**-3 is the included barriers from the head and 10**3 converts Sv to mSv
    return H_leak_uns # (mSv/wk)

def convert_to_meters(foot,inches):
    m = (foot * 12 * 2.54 + inches * 2.54) * 10e-3
    return m

def verify_2barrier(P, T, thickness,d,tvl_6mv_ps,tvl_18mv_ps,a_6mv,a_18mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param tvl_18mv_ps: TVL for 18 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    param a_18mv: scatter fraction for 18 MV
    '''
  
    H_l_6mv = H_leak_unshielded(1990, d) * B2(thickness, 34, 29)
    H_l_18mv = H_leak_unshielded(250, d) * B2(thickness, 36, 34)

    H_ps_6mv = H_ps_unshielded(520, d, 1, a_6mv, 1600) * B1(thickness, tvl_6mv_ps)
    H_ps_18mv = H_ps_unshielded(250, d, 1, a_18mv, 1600) * B1(thickness, tvl_18mv_ps)

    H_total = H_ps_6mv + H_ps_18mv + H_l_6mv + H_l_18mv
    
    if H_total < P /T:
        print("PASS")
    else:
        print("FAIL")
    print("Total dose: ", H_total, "mSv/wk")
    print("Dose limit: ", P / T, "mSv/wk")
    print(f"Thickness used: {thickness} cm = {thickness /2.54} inches")
    print(f"distance from target to 0.3m beyond distal surface: {d} m")

# Important note: thickness must be in centimeters.

def verify_2barrier_vault2(P, T, thickness,d,tvl_6mv_ps,a_6mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    '''
  
    H_l_6mv = H_leak_unshielded(1435.5, d) * B2(thickness, 34, 29)

    H_ps_6mv = H_ps_unshielded(319, d, 1, a_6mv, 400) * B1(thickness, tvl_6mv_ps)

    H_total = H_ps_6mv + H_l_6mv 
    if H_total < P /T:
        print("PASS")
    else:
        print("FAIL")
    print("Total dose: ", H_total, "mSv/wk")
    print("Dose limit: ", P / T, "mSv/wk")
    print(f"Thickness used: {thickness} cm = {thickness /2.54} inches")

# Important note: thickness must be in centimeters.

# Verification of the point just outside the door

In [70]:
# P = 0.1 mSv/wk
# T = 1 
# NCRP lists occupancy outside a door as 1/8, but we can be conservtative and use T=1
thickness_IMW = convert_to_meters(2,9.5) * 100 # convert to cm
test = thickness_IMW / math.cos(math.radians(23.54)) # corrected thickness for 23.54 degrees

verify_2barrier(0.1,1, thickness_IMW, 7.578673946, 23, 27, 1.39e-3,8.64e-4)

PASS
Total dose:  0.0963030524558548 mSv/wk
Dose limit:  0.1 mSv/wk
Thickness used: 85.09 cm = 33.5 inches
distance from target to 0.3m beyond distal surface: 7.578673946 m


# Vault #1 Wall A, Secondary Barrier

In [71]:
# Worst case scenario of 33.37 deg barely grazing the edge of primary barrier
# 8.57258 m distance
#2'-3" = 2.25 ft = 68.58 cm
verify_2barrier(0.02,(1/40),68.58,8.57258702,26,32,2.77e-3,2.53e-3)

PASS
Total dose:  0.6394931153452919 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk
Thickness used: 68.58 cm = 27.0 inches
distance from target to 0.3m beyond distal surface: 8.57258702 m


# Vault #1 Wall C, Secondary Barrier

In [72]:
# Worst case scenario of 33.76 deg barely grazing the edge of primary barrier
verify_2barrier(0.1,(1/2),convert_to_meters(2,11) * 100,convert_to_meters(28,3.01894),26,32,2.77e-3,2.53e-3)

PASS
Total dose:  0.13016439261202056 mSv/wk
Dose limit:  0.2 mSv/wk
Thickness used: 88.9 cm = 35.0 inches
distance from target to 0.3m beyond distal surface: 8.611081076000001 m


# Vault #1 Wall B, Secondary Barrier

In [73]:
# Worst case scenario of 90 deg scattering
verify_2barrier(0.02,1,106.9,convert_to_meters(20,11.81056),17,19,4.26e-4,1.89e-4)

PASS
Total dose:  0.01994554665683441 mSv/wk
Dose limit:  0.02 mSv/wk
Thickness used: 106.9 cm = 42.08661417322835 inches
distance from target to 0.3m beyond distal surface: 6.395988224 m


# Vault # 2 Wall C, Secondary Barrier

In [78]:
# Worst case scenario of 23.99 deg scatter
# perpendicular thickness used instead of olblique thickness
verify_2barrier_vault2(0.1,(1/2), 88.9, convert_to_meters(28,9.11344,), 34, 6.73e-3)
print("-------")
verify_2barrier_vault2(0.1,(1/2), 94.615, convert_to_meters(28,9.11344,), 34, 6.73e-3)
print("-------")
verify_2barrier_vault2(0.1,(1/2), convert_to_meters(3,2.30809) * 100, convert_to_meters(28,9.11344,), 34, 6.73e-3)
print("Above calculation uses the oblique thickness.")

PASS
Total dose:  0.09173865344551867 mSv/wk
Dose limit:  0.2 mSv/wk
Thickness used: 88.9 cm = 35.0 inches
-------
PASS
Total dose:  0.061249113192355945 mSv/wk
Dose limit:  0.2 mSv/wk
Thickness used: 94.615 cm = 37.25 inches
-------
PASS
Total dose:  0.0506659809864869 mSv/wk
Dose limit:  0.2 mSv/wk
Thickness used: 97.3025486 cm = 38.30809 inches
Above calculation uses the oblique thickness.


Our secondary barrier in Vault #1, 2'-9.5" = 88.9 cm, is not enough to shield this secondary beam. Thus, we will use 94.615 cm = 37 1/4" in the secondary barrier.

This means that we must add 3.75" of extra concrete into the Vault #2.

In [79]:
convert_to_meters(20,11.80982)

6.395969428000001